### Зависимости и константы

In [1]:
import pandas as pd
import numpy as np
from random import randint, choice, choices
import json

from IPython.core.display import deepcopy
from collections import Counter

In [2]:
DAYS = 5
HOURS_PER_DAY = 8
SLOTS_PER_DAY = HOURS_PER_DAY * 2
STAFF_NUM = 8

### Чтение данных

In [3]:
def fix_index(index):
    return str(index).encode('cp1251').decode('utf-8')

def fix_column_name(column):
    return str(column).encode('cp1251').decode('utf-8')


distance = pd.read_excel('Копия output_v3.xlsx')
distance = distance.set_index(0)
distance = distance.rename(columns=fix_column_name)
distance.index = distance.index.map(fix_index)
distance.drop_duplicates(inplace=True)
distance

,"Краснодар, ул. Ставропольская, д. 140","Краснодар, ул. им. Максима Горького, д. 128","Краснодар, ул. им. Дзержинского, д. 100","Краснодар, ул. Красноармейская, д. 126","Краснодар, х. Ленина, п/о. 37","Краснодар, тер. Пашковский жилой массив, ул. Крылатая, д. 2","Краснодар, ул. Восточно-Кругликовская, д. 64/2","Краснодар, ул. Красных Партизан, д. 439","Краснодар, ул. Таманская, д. 153 к. 3, кв. 2","Краснодар, ул. им. Дзержинского, д. 165",...,"Краснодар, п. Березовый, ул. Целиноградская, д. 6/1","Краснодар, ул. им. Дзержинского, д. 100.2","Краснодар, ул. Российская, д. 418","Краснодар, ул. им. Володи Головатого, д. 313","Краснодар, ул. Красная, д. 145","Краснодар, ул. Красная, д. 154","Краснодар, Красная, д. 139","Краснодар, В.Н. Мачуги, 41","Краснодар, Красных Партизан, 321","Краснодар, ул. Зиповская, д. 1"
0,,,,,,,,,,,,,,,,,,,,,
"Краснодар, ул. Ставропольская, д. 140",0.000000,2847.127124,7473.813374,3347.673418,26567.926263,13197.185140,4838.963493,5936.052294,1043.629060,6818.843185,...,10923.341445,7473.813374,5798.717250,3696.928465,3725.193424,3476.029309,3688.902992,7554.862617,7473.719505,4104.401170
"Краснодар, ул. им. Максима Горького, д. 128",2847.127124,0.000000,6265.570381,725.357331,29237.390518,15963.726915,6093.621861,3160.872987,3738.627496,5224.979652,...,10240.163697,6265.570381,6188.078303,995.555137,1453.167105,747.698231,1315.432509,10343.787574,4643.823111,2849.679461
"Краснодар, ул. им. Дзержинского, д. 100",7473.813374,6265.570381,0.000000,5615.279711,29299.345076,17288.536320,5492.117823,5340.710794,7529.520950,1318.285239,...,4111.067071,0.000000,3748.472229,5539.355445,4856.845317,5709.530627,5017.900891,12450.743726,6263.311642,3520.790484
"Краснодар, ул. Красноармейская, д. 126",3347.673418,725.357331,5615.279711,0.000000,29483.969139,16296.541185,5979.110960,2588.564558,4139.407146,4534.906879,...,9634.893161,5615.279711,5857.027571,351.102253,760.405663,191.375254,606.887384,10710.676272,4147.159703,2306.120823
"Краснодар, х. Ленина, п/о. 37",26567.926263,29237.390518,29299.345076,29483.969139,0.000000,13529.898320,24263.366135,31814.689993,25561.430272,29926.830589,...,28704.802404,29299.345076,25643.638318,29810.700852,29447.106267,29662.348162,29513.923465,19110.060169,33429.854446,28526.505596
"Краснодар, тер. Пашковский жилой массив, ул. Крылатая, д. 2",13197.185140,15963.726915,17288.536320,16296.541185,13529.898320,0.000000,11831.612718,18763.637045,12231.180608,17596.350134,...,18019.425030,17288.536320,13551.556668,16637.883529,16372.389992,16463.833978,16414.397940,5647.964131,20380.507705,15710.814607
"Краснодар, ул. Восточно-Кругликовская, д. 64/2",4838.963493,6093.621861,5492.117823,5979.110960,24263.366135,11831.612718,0.000000,7813.477599,4120.566271,5781.076252,...,7354.853798,5492.117823,1893.595652,6220.389877,5640.919680,6169.808758,5759.390438,7012.917172,9370.945704,4370.923059
"Краснодар, ул. Красных Партизан, д. 439",5936.052294,3160.872987,5340.710794,2588.564558,31814.689993,18763.637045,7813.477599,0.000000,6705.789069,4026.100826,...,9406.916701,5340.710794,7117.326472,2239.176197,2391.392822,2465.055370,2353.877440,13224.873055,1620.116088,3458.476251
"Краснодар, ул. Таманская, д. 153 к. 3, кв. 2",1043.629060,3738.627496,7529.520950,4139.407146,25561.430272,12231.180608,4120.566271,6705.789069,0.000000,7044.919437,...,10698.146722,7529.520950,5346.216853,4489.598324,4383.208147,4290.628056,4379.875691,6605.272245,8285.513570,4401.280785


In [4]:
distance.loc['Краснодар, ул. Красная, д. 154', 'Краснодар, ул. Ставропольская, д. 140']

3476.0293094100857

In [5]:
data = pd.read_excel('Копия ДатаСет_Финал.xlsx', sheet_name='Входные данные для анализа')
data.dropna(inplace=True, axis=0)
data.loc[22, 'Адрес точки, г. Краснодар'] = 'ул. им. Героя Аверкиева А.А., д. 8'
data

,№ точки,"Адрес точки, г. Краснодар",Когда подключена точка?,Карты и материалы доставлены?,Кол-во дней после выдачи последней карты,Кол-во одобренных заявок,Кол-во выданных карт
0,1,"ул. Ставропольская, д. 140",вчера,нет,0.0,0.0,0.0
1,2,"ул. им. Максима Горького, д. 128",давно,да,3.0,15.0,3.0
2,3,"ул. им. Дзержинского, д. 100",давно,да,3.0,9.0,1.0
3,4,"ул. Красноармейская, д. 126",давно,да,0.0,38.0,23.0
4,5,"х. Ленина, п/о. 37",давно,нет,0.0,14.0,0.0
5,6,"тер. Пашковский жилой массив, ул. Крылатая, д. 2",давно,да,12.0,19.0,1.0
6,7,"ул. Восточно-Кругликовская, д. 64/2",давно,да,27.0,19.0,12.0
7,8,"ул. Красных Партизан, д. 439",давно,да,33.0,84.0,63.0
8,9,"ул. Таманская, д. 153 к. 3, кв. 2",давно,да,2.0,15.0,1.0
9,10,"ул. им. Дзержинского, д. 165",давно,да,0.0,19.0,0.0


In [6]:
hashmap_address = {}

for point, address in zip(data['№ точки'], data['Адрес точки, г. Краснодар']):
    hashmap_address[point] = 'Краснодар, ' + address

In [7]:
staff = pd.read_excel('Копия ДатаСет_Финал.xlsx', sheet_name='Справочник сотрудников')
staff

,ФИО,Адрес локации,Грейд
0,Дерягин Никита Владимирович,"Краснодар, Красная, д. 139",Синьор
1,Петрошев Валерий Павлович,"Краснодар, Красная, д. 139",Мидл
2,Евдокимов Давид Тихонович,"Краснодар, Красная, д. 139",Джун
3,Андреев Гордий Данилович,"Краснодар, В.Н. Мачуги, 41",Синьор
4,Иванов Адам Федорович,"Краснодар, В.Н. Мачуги, 41",Мидл
5,Бобылёв Ипполит Альбертович,"Краснодар, В.Н. Мачуги, 41",Джун
6,Беляева Евгения Антоновна,"Краснодар, Красных Партизан, 321",Мидл
7,Николаев Азарий Платонович,"Краснодар, Красных Партизан, 321",Джун


### Добавление приоритетности задач

In [8]:
res = []
for row in data.values.tolist():
    if (row[-3] > 7 and row[-2] > 0) or (row[-3] > 14):
        res.append('Выезд на точку для стимулирования выдач')
    elif row[-1] > 0 and (row[-1] / row[-2] < 0.5):
        res.append('Обучение агента')
    elif row[2] == 'вчера' or row[3] == 'нет':
        res.append('Доставка карт и материалов')
    else:
        res.append('-')

In [9]:
data_task = data.copy()
data_task['Задача'] = res
data_task = data_task.loc[data_task['Задача'] != '-']
data_task.loc[data_task['Задача'] == 'Выезд на точку для стимулирования выдач', 'Приоретет'] = 'Высокий'
data_task.loc[data_task['Задача'] == 'Обучение агента', 'Приоретет'] = 'Средний'
data_task.loc[data_task['Задача'] == 'Доставка карт и материалов', 'Приоретет'] = 'Низкий'
data_task

,№ точки,"Адрес точки, г. Краснодар",Когда подключена точка?,Карты и материалы доставлены?,Кол-во дней после выдачи последней карты,Кол-во одобренных заявок,Кол-во выданных карт,Задача,Приоретет
0,1,"ул. Ставропольская, д. 140",вчера,нет,0.0,0.0,0.0,Доставка карт и материалов,Низкий
1,2,"ул. им. Максима Горького, д. 128",давно,да,3.0,15.0,3.0,Обучение агента,Средний
2,3,"ул. им. Дзержинского, д. 100",давно,да,3.0,9.0,1.0,Обучение агента,Средний
4,5,"х. Ленина, п/о. 37",давно,нет,0.0,14.0,0.0,Доставка карт и материалов,Низкий
5,6,"тер. Пашковский жилой массив, ул. Крылатая, д. 2",давно,да,12.0,19.0,1.0,Выезд на точку для стимулирования выдач,Высокий
6,7,"ул. Восточно-Кругликовская, д. 64/2",давно,да,27.0,19.0,12.0,Выезд на точку для стимулирования выдач,Высокий
7,8,"ул. Красных Партизан, д. 439",давно,да,33.0,84.0,63.0,Выезд на точку для стимулирования выдач,Высокий
8,9,"ул. Таманская, д. 153 к. 3, кв. 2",давно,да,2.0,15.0,1.0,Обучение агента,Средний
10,11,"ст-ца. Елизаветинская, ул. Широкая, д. 260",давно,да,15.0,29.0,15.0,Выезд на точку для стимулирования выдач,Высокий
11,12,"ул. им. Тургенева, д. 174, 1 этаж",вчера,нет,0.0,0.0,0.0,Доставка карт и материалов,Низкий


In [10]:
def get_tasks():
    tasks = [[id, t, p] for id, t, p in zip(data_task['№ точки'], data_task['Задача'].values.tolist(), data_task['Приоретет'].values.tolist())]

    task_type = {'Доставка карт и материалов': (3, 1.5), 'Обучение агента' : (2, 2), 'Выезд на точку для стимулирования выдач': (1, 4)}
    priority = {'Низкий': 1, 'Средний': 2, 'Высокий':3}

    for i in range(len(tasks)):
        tasks[i].append(int(task_type[tasks[i][1]][1] / 0.5))
        tasks[i][1] = task_type[tasks[i][1]][0]
        tasks[i][2] = priority[tasks[i][2]]
    return tasks

### Генерация генома и популяции

In [11]:
def random_position(cur_task):
    duration = cur_task[-1]
    day = randint(0, DAYS - 1)
    worker = randint(0, STAFF_NUM - 1)
    time = randint(0, SLOTS_PER_DAY - duration)
    position = day * STAFF_NUM * SLOTS_PER_DAY + worker * SLOTS_PER_DAY + time
    return position

In [12]:
def generate_genome():
    genome = [[] for _ in range(DAYS * SLOTS_PER_DAY * STAFF_NUM)]
    hashmap = {}
    position_list = []
    s=0
    for cur_task in get_tasks():
        duration = cur_task[-1]
        position = random_position(cur_task)

        position_list.append(position)
        hashmap[cur_task[0]] = cur_task[1:], position

        for i in range(position, position + duration):
            genome[i].append(cur_task[0])

    return genome, hashmap

In [13]:
def generate_population():
    population = []
    for _ in range(100):
        genome = generate_genome()
        population.append(genome)
    return population

In [14]:
def schedule(genome):
    day, worker, slot = 0,0,0
    pos = 0
    for day_pos in range(0,640,640//5):
        day += 1
        worker = 0
        print(f'================DAY{day}================')
        for worker_pos in range(day_pos, day_pos + 640//5, 16):
            worker += 1
            print(f'----------WORKER{worker}----------')
            for i in range(16):
                s = ''
                pos = worker_pos + i
                key = genome[0][pos]
                s += str(key)
                for i in range(len(key)):
                    s += ' '
                    s+=str(genome[1][key[i]])
                print(s)

In [15]:
# Тестовые образцы
g = generate_genome()
p = generate_population()

In [16]:
print(f'Общее кол-во ячеек: {SLOTS_PER_DAY * DAYS * STAFF_NUM}')
print(f'Количество заполненных ячеек: {sum([i[3] for i in get_tasks()])}')

Общее кол-во ячеек: 640
Количество заполненных ячеек: 169


In [17]:
g[0]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [9],
 [9],
 [9],
 [9],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [28],
 [28],
 [28],
 [28],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [39],
 [39],
 [39],
 [],
 [],
 [],
 [],
 [15],
 [15],
 [15],
 [15],
 [],
 [],
 [],
 [29],
 [29],
 [29],
 [29],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [30],
 [30],
 [30],
 [30],
 [30],
 [30],
 [30],
 [30],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [25],
 [25],
 [25],
 [],
 [],
 [],
 [],
 [],
 [],

In [18]:
schedule(g)

================DAY1================
----------WORKER1----------
[]
[]
[]
[]
[]
[]
[]
[9] ([2, 2, 4], 7)
[9] ([2, 2, 4], 7)
[9] ([2, 2, 4], 7)
[9] ([2, 2, 4], 7)
[]
[]
[]
[]
[]
----------WORKER2----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER3----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER4----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER5----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER6----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER7----------
[]
[]
[]
[]
[]
[]
[28] ([2, 2, 4], 102)
[28] ([2, 2, 4], 102)
[28] ([2, 2, 4], 102)
[28] ([2, 2, 4], 102)
[]
[]
[]
[]
[]
[]
----------WORKER8----------
[]
[]
[39] ([3, 1, 3], 114)
[39] ([3, 1, 3], 114)
[39] ([3, 1, 3], 114)
[]
[]
[]
[]
[15] ([2, 2, 4], 121)
[15] ([2, 2, 4], 121)
[15] ([2, 2, 4], 121)
[15] ([2, 2, 4], 121)
[]
[]
[]
================DAY2================
----------WORKER1----------
[29] ([2, 2, 4], 

### Реализация скрещивания

In [19]:
def genome_from_hash(hashmap):
    keys = list(hashmap.keys())
    genome = [[] for _ in range(DAYS * SLOTS_PER_DAY * STAFF_NUM)]
    for key in keys:
        duration = hashmap[key][0][-1]
        position = hashmap[key][1]

        for i in range(position, position + duration):
            genome[i].append(key)

    return genome

In [20]:
def crossover(parent1, parent2):
    keys = list(parent1[1].keys())
    crossover_point = randint(1, len(keys))
    offspring1_hash = {}
    offspring2_hash = {}

    for key in keys[ : crossover_point]:
        offspring1_hash[key] = parent1[1][key]
    for key in keys[crossover_point : ]:
        offspring1_hash[key] = parent2[1][key]

    offspring1_genome = genome_from_hash(offspring1_hash)

    for key in keys[ : crossover_point]:
        offspring2_hash[key] = parent2[1][key]
    for key in keys[crossover_point : ]:
        offspring2_hash[key] = parent1[1][key]

    offspring2_genome = genome_from_hash(offspring2_hash)

    return (offspring1_genome, offspring1_hash), (offspring2_genome, offspring2_hash)

In [21]:
parent1 = generate_genome()
parent2 = generate_genome()

In [22]:
off1, off2 = crossover(parent1, parent2)
schedule(off1)

================DAY1================
----------WORKER1----------
[]
[]
[]
[]
[]
[]
[]
[]
[40] ([2, 2, 4], 8)
[40] ([2, 2, 4], 8)
[40] ([2, 2, 4], 8)
[40] ([2, 2, 4], 8)
[]
[]
[]
[]
----------WORKER2----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER3----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER4----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER5----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER6----------
[]
[]
[]
[]
[]
[]
[36] ([2, 2, 4], 86)
[36] ([2, 2, 4], 86)
[36] ([2, 2, 4], 86)
[36] ([2, 2, 4], 86)
[]
[]
[]
[]
[]
[]
----------WORKER7----------
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
----------WORKER8----------
[]
[]
[]
[]
[]
[2] ([2, 2, 4], 117)
[2, 42] ([2, 2, 4], 117) ([2, 2, 4], 118)
[2, 42] ([2, 2, 4], 117) ([2, 2, 4], 118)
[2, 42] ([2, 2, 4], 117) ([2, 2, 4], 118)
[42] ([2, 2, 4], 118)
[]
[]
[]
[]
[]
[]
================DAY2================
----------WORKER1-------

### Реализация мутации

In [23]:
def mutation(genome):
    random_task = choice(list(genome[1].keys()))
    pos = random_position(genome[1][random_task][0])
    mutated_hash = {key : value for key, value in zip(list(genome[1].keys()), list(genome[1].values()))}
    mutated_hash[random_task] = (mutated_hash[random_task][0], pos)
    mutated_genome = genome_from_hash(mutated_hash)
    
    return mutated_genome, mutated_hash

In [24]:
mutated_genome = mutation(g)

### Реализация штрафов

In [25]:
def get_distance(loc1, loc2):
    meters = distance.loc[loc1, loc2]
    if meters == 0:
        return 0
    if meters > 15000:
        return int(np.ceil(meters / 1000 / 40 * 60 / 30))
    else:
        return int(np.ceil(meters / 1000 / 20 * 60 / 30))

In [26]:
task_type = {'Доставка карт и материалов': (3, 1.5), 'Обучение агента' : (2, 2), 'Выезд на точку для стимулирования выдач': (1, 4)}
priority = {'Низкий': 1, 'Средний': 2, 'Высокий':3}
employee_grade = dict(zip(list(staff.index), staff['Грейд']))
grade_tasks = {'Синьор': [task_type['Доставка карт и материалов'][0],
                          task_type['Обучение агента'][0],
                          task_type['Выезд на точку для стимулирования выдач'][0]],
             'Мидл': [task_type['Доставка карт и материалов'][0], task_type['Обучение агента'][0]],
             'Джун': [task_type['Доставка карт и материалов'][0]]}
employee_address = dict(zip(list(staff.index), staff['Адрес локации']))

def append_time(genome):
    score = 0
    new_genome = deepcopy(genome)
    for key in list(new_genome[1].keys()):

        # duration_ab = 0
        task = new_genome[1][key]
        position = task[1]
        worker_start = position // 128 * STAFF_NUM * SLOTS_PER_DAY + (position % 128) // 16 * SLOTS_PER_DAY
        flag = 0

        for i in range(position-1, worker_start-1, -1):
            if new_genome[0][i] != []:
                flag = 1
                for z in range(len(new_genome[0][i])):
                    duration_ab = get_distance(hashmap_address[new_genome[0][i][z]], hashmap_address[new_genome[0][position][0]])
        if flag == 0:
            duration_ab = get_distance(employee_address[(position % 128) // 16], hashmap_address[new_genome[0][position][0]])


        for p in range(position - 1, position - 1 - duration_ab, -1):
            new_genome[0][p].append(key)
            if p < worker_start:
                score += 720
    return new_genome, score

def fitness_genome(genome):
    fit_genome = deepcopy(genome)
    hashmap = fit_genome[1]
    fit_genome, score = append_time(fit_genome)

    for position, gene in enumerate(fit_genome[0]):
        if len(gene) > 1:
            score += 800
        if gene == []:
            score += 640 - position
        if gene != []:
            for i in range(len(gene)):
                if hashmap[gene[i]][0][0] not in grade_tasks[employee_grade[(position % 128) // 16]]:
                    score += 640
                if hashmap[gene[i]][0][1] == 3:
                    score += position
                elif hashmap[gene[i]][0][1] == 2:
                    score += position // 1.5

    return score

### Эволюция

In [27]:
def create_top_population(population):
    hashmap_score = {}
    for idx, genom in enumerate(population):
        hashmap_score[idx] = fitness_genome(genom)
    hashmap_score = {k: v for k, v in sorted(hashmap_score.items(), key=lambda item: item[1])}
    top_population = []
    hashmap_score_keys = list(hashmap_score.keys())
    for i in range(len(hashmap_score)//2):
        top_population.append(append_time(population[hashmap_score_keys[i]])[0])  #######
    return top_population, hashmap_score


def evolution(n:int, best_score = 10000000):
    population = generate_population()
    top_population, _ = create_top_population(population)

    for i in range(n):
        for genom1, genom2 in zip(top_population[::2], top_population[1::2]):
            new_genom1, new_genom2 = crossover(genom1, genom2)
            new_genom1_mutation = mutation(new_genom1)
            new_genom2_mutation = mutation(new_genom2)
            top_population.extend([new_genom1_mutation, new_genom2_mutation])

        top_population, hashmap_score = create_top_population(top_population)
        max_score = list(hashmap_score.values())[0]
        if max_score < best_score:
            best_score = max_score
            print(best_score)
    return top_population[0]

In [28]:
top_population = evolution(1000) # Поставил 1000 лишь для примера, необходимо ставить от 20.000 до 30.000

231242.0
228240.0
209594.0
206823.0
206751.0
203351.0
199895.0
192011.0
191247.0
185222.0
183561.0
179974.0
178350.0
174933.0
171715.0
168259.0
167854.0
166598.0
166505.0
165179.0
161862.0
161220.0
160492.0
154447.0
150951.0
147223.0
146886.0
145925.0
145722.0
144918.0
144883.0
142889.0
142033.0
141215.0
140864.0
140613.0
140050.0
139187.0
139185.0
137447.0
137364.0
137328.0
135755.0
135743.0
135463.0
134466.0
134202.0
134018.0
133947.0
133189.0
133152.0
132972.0
132926.0
132785.0
132241.0
130737.0
129350.0
128820.0


In [33]:
best_genome = top_population
schedule(best_genome)

================DAY1================
----------WORKER1----------
[]
[28] ([2, 2, 4], 2)
[28] ([2, 2, 4], 2)
[28] ([2, 2, 4], 2)
[28] ([2, 2, 4], 2)
[28] ([2, 2, 4], 2)
[37] ([1, 3, 8], 7)
[37] ([1, 3, 8], 7)
[37] ([1, 3, 8], 7)
[37] ([1, 3, 8], 7)
[37] ([1, 3, 8], 7)
[37] ([1, 3, 8], 7)
[37] ([1, 3, 8], 7)
[37] ([1, 3, 8], 7)
[37] ([1, 3, 8], 7)
[]
----------WORKER2----------
[40] ([2, 2, 4], 17)
[40] ([2, 2, 4], 17)
[40] ([2, 2, 4], 17)
[40] ([2, 2, 4], 17)
[40] ([2, 2, 4], 17)
[]
[]
[29] ([2, 2, 4], 24)
[29] ([2, 2, 4], 24)
[29] ([2, 2, 4], 24)
[29] ([2, 2, 4], 24)
[29, 22] ([2, 2, 4], 24) ([2, 2, 4], 28)
[22] ([2, 2, 4], 28)
[22] ([2, 2, 4], 28)
[22] ([2, 2, 4], 28)
[22] ([2, 2, 4], 28)
----------WORKER3----------
[]
[]
[]
[16] ([3, 1, 3], 37)
[16] ([3, 1, 3], 37)
[16] ([3, 1, 3], 37)
[16] ([3, 1, 3], 37)
[16] ([3, 1, 3], 37)
[1] ([3, 1, 3], 41)
[1] ([3, 1, 3], 41)
[1] ([3, 1, 3], 41)
[1] ([3, 1, 3], 41)
[]
[]
[]
[41] ([2, 2, 4], 49)
----------WORKER4----------
[41] ([2, 2, 4], 49)


### Выдача данных для Бэка

In [48]:
def get_tasks_day(top_genome):
    task_type_hash = {3: 'Доставка карт и материалов', 2: 'Обучение агента',
                      1: 'Выезд на точку для стимулирования выдач'}
    one_day_g = top_genome[0][0:128]
    work_time = Counter([x for xs in one_day_g for x in xs if x != []])
    json_dict = {}
    idx_for_drop = []
    for i in range(0, 128, 16):
        staff_address = []
        for j in range(i, i + 16):
            if one_day_g[j] != []:
                for task in range(len(one_day_g[j])):
                    address = hashmap_address[one_day_g[j][task]]
                    if address not in staff_address:
                        staff_address.append(address)
                        task_type = task_type_hash[top_genome[1][one_day_g[j][task]][0][0]]
                        staff_name = i // 16
                        if staff_name not in json_dict.keys():
                            json_dict[staff_name] = [[address, task_type, work_time[one_day_g[j][task]] * 30]]
                        else:
                            json_dict[staff_name].append([address, task_type, work_time[one_day_g[j][task]] * 30])
                        idx_for_drop.append(one_day_g[j][task])
    return json.dumps(json_dict), idx_for_drop, json_dict

In [49]:
a, idx_for_drop, json_dict = get_tasks_day(best_genome)
print(a)
print()
print(idx_for_drop)
print()
print(json_dict)

{"0": [["\u041a\u0440\u0430\u0441\u043d\u043e\u0434\u0430\u0440, \u0443\u043b. \u0421\u0435\u0432\u0435\u0440\u043d\u0430\u044f, \u0434. 524", "\u041e\u0431\u0443\u0447\u0435\u043d\u0438\u0435 \u0430\u0433\u0435\u043d\u0442\u0430", 150], ["\u041a\u0440\u0430\u0441\u043d\u043e\u0434\u0430\u0440, \u0443\u043b. \u041a\u0440\u0430\u0441\u043d\u0430\u044f, \u0434. 149", "\u0412\u044b\u0435\u0437\u0434 \u043d\u0430 \u0442\u043e\u0447\u043a\u0443 \u0434\u043b\u044f \u0441\u0442\u0438\u043c\u0443\u043b\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u044f \u0432\u044b\u0434\u0430\u0447", 270]], "1": [["\u041a\u0440\u0430\u0441\u043d\u043e\u0434\u0430\u0440, \u0443\u043b. \u0420\u043e\u0441\u0441\u0438\u0439\u0441\u043a\u0430\u044f, \u0434. 418", "\u041e\u0431\u0443\u0447\u0435\u043d\u0438\u0435 \u0430\u0433\u0435\u043d\u0442\u0430", 150], ["\u041a\u0440\u0430\u0441\u043d\u043e\u0434\u0430\u0440, \u0443\u043b. \u0438\u043c. \u041a\u0438\u0440\u0438\u043b\u043b\u0430 \u0420\u043e\u0441\u0441\u0438\u04

In [50]:
def create_newdata():
    new_data = data_task[~data_task['№ точки'].isin(idx_for_drop)].copy()
    new_data.reset_index(drop=True, inplace=True)
    new_data['Приоретет'] = 'Высокий'
    return new_data

In [51]:
new_data = create_newdata()
new_data

,№ точки,"Адрес точки, г. Краснодар",Когда подключена точка?,Карты и материалы доставлены?,Кол-во дней после выдачи последней карты,Кол-во одобренных заявок,Кол-во выданных карт,Задача,Приоретет
0,2,"ул. им. Максима Горького, д. 128",давно,да,3.0,15.0,3.0,Обучение агента,Высокий
1,3,"ул. им. Дзержинского, д. 100",давно,да,3.0,9.0,1.0,Обучение агента,Высокий
2,5,"х. Ленина, п/о. 37",давно,нет,0.0,14.0,0.0,Доставка карт и материалов,Высокий
3,6,"тер. Пашковский жилой массив, ул. Крылатая, д. 2",давно,да,12.0,19.0,1.0,Выезд на точку для стимулирования выдач,Высокий
4,7,"ул. Восточно-Кругликовская, д. 64/2",давно,да,27.0,19.0,12.0,Выезд на точку для стимулирования выдач,Высокий
5,8,"ул. Красных Партизан, д. 439",давно,да,33.0,84.0,63.0,Выезд на точку для стимулирования выдач,Высокий
6,9,"ул. Таманская, д. 153 к. 3, кв. 2",давно,да,2.0,15.0,1.0,Обучение агента,Высокий
7,11,"ст-ца. Елизаветинская, ул. Широкая, д. 260",давно,да,15.0,29.0,15.0,Выезд на точку для стимулирования выдач,Высокий
8,12,"ул. им. Тургенева, д. 174, 1 этаж",вчера,нет,0.0,0.0,0.0,Доставка карт и материалов,Высокий
9,19,"ул. Зиповская, д. 1",давно,да,6.0,32.0,9.0,Обучение агента,Высокий
